In [1]:
import pandas as pd
import numpy as np
import math
import swifter

from model_functions import Glicko, Processing, DataChange
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV, cross_validate
from sklearn.dummy import DummyClassifier
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

In [2]:
# Files
# DATASET = 'dataset/unused_datasets/atp_matches_2023.csv'
DATASET = 'dataset_WL_Batch2.csv'
COUNTRY_CODES = 'dataset/ioc_iso.csv'
TOURNAMENTS = 'dataset/tournaments.csv'
IDS = 'dataset/player_ids.csv'

In [4]:
df = pd.read_csv(DATASET, dtype={
    'tourney_level': str}, keep_default_na=False)
df.index = range(df.shape[0])

# Create initial columns
df['tourney_datetime'] = pd.to_datetime(df['tourney_date'], format='%Y%m%d').dt.date
df['tourney_date_int'] = df['tourney_date'].astype('int32')

# Read supporting datasets
iso_df = pd.read_csv(COUNTRY_CODES, keep_default_na=False)
tournament_loc_df = pd.read_csv(TOURNAMENTS)
id_df = pd.read_csv(IDS, encoding = "ISO-8859-1")

C:\Users\ivan_\AppData\Local\Temp\ipykernel_14276\3748443927.py:1: DtypeWarning: Columns (31,32,33,34,35,36,37,38,39,40,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(DATASET, dtype={


In [11]:
for index, row in df.iterrows():
    if row['p1_qual/chal_losses_alltime'] == '':
        print(row['p1_name'])

Diego Schwartzman
Miomir Kecmanovic
Alexandre Muller
Albert Ramos
Marton Fucsovics
Gael Monfils
Alvaro Lopez San Martin


In [12]:
frances_df = df[(df['p1_name'] == 'Diego Schwartzman') | (df['p2_name'] == 'Diego Schwartzman')]
frances_df.index = range(frances_df.shape[0])
display(frances_df)

,tourney_date,tourney_level,p1_name,p2_name,p1_age,p2_age,p1_iso,p2_iso,winner,winner_name,...,p1_tour_wins_last_year,p1_tour_losses_last_year,p1_tour_wins_alltime,p1_tour_losses_alltime,p1_qual/chal_wins_last_sem,p1_qual/chal_losses_last_sem,p1_qual/chal_wins_last_year,p1_qual/chal_losses_last_year,p1_qual/chal_wins_alltime,p1_qual/chal_losses_alltime
0,20090511,S,Diego Schwartzman,Juan Vazquez Valenzuela,16.7,18.0,AR,AR,1,Juan Vazquez Valenzuela,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,20090615,S,Juan Pablo Villar,Diego Schwartzman,26.9,16.8,AR,AR,0,Juan Pablo Villar,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,7.0,16.0
2,20090727,S,Gustavo Junqueira De Andrade,Diego Schwartzman,19.8,16.9,BR,AR,0,Gustavo Junqueira De Andrade,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,20090907,S,Diego Schwartzman,Guido Andreozzi,17.0,18.0,AR,AR,1,Guido Andreozzi,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,20090914,S,Federico Cavallero,Diego Schwartzman,21.5,17.0,AR,AR,1,Diego Schwartzman,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
874,20230417,A,Diego Schwartzman,Yibing Wu,30.6,23.5,AR,CN,0,Diego Schwartzman,...,19.0,29.0,272.0,222.0,0.0,1.0,0.0,1.0,124.0,69.0
875,20230424,M,Diego Schwartzman,Hugo Grenier,30.6,27.0,AR,FR,1,Hugo Grenier,...,17.0,28.0,273.0,222.0,0.0,1.0,0.0,1.0,124.0,69.0
876,20230501,C,Diego Schwartzman,Mattia Bellucci,30.6,21.8,AR,IT,1,Mattia Bellucci,...,17.0,29.0,273.0,223.0,0.0,1.0,0.0,1.0,124.0,69.0
877,20230508,M,Diego Schwartzman,Matteo Arnaldi,30.7,22.2,AR,IT,1,Matteo Arnaldi,...,16.0,28.0,273.0,223.0,0.0,2.0,0.0,2.0,124.0,70.0


In [4]:
import numpy as np
import pandas as pd
np.random.seed(125)
N = 40000000
df = pd.DataFrame({'A':np.random.randint(10, size=N)})

In [9]:
def bex(x):
    return x[(x.A.values > 5) & (x.A.values < 9)]
bex(df)

,A
1,7
3,6
6,6
14,7
16,7
...,...
39999993,7
39999994,8
39999995,6
39999996,6


In [5]:
df

,A
0,2
1,7
2,3
3,6
4,0
...,...
39999995,6
39999996,6
39999997,6
39999998,0


In [13]:
frances_df.loc[878]

tourney_date                              20230522
tourney_level                                    A
p1_name                          Diego Schwartzman
p2_name                          Brandon Nakashima
p1_age                                        30.7
p2_age                                        21.7
p1_iso                                          AR
p2_iso                                          US
winner                                           1
winner_name                      Brandon Nakashima
year                                          2023
month                                            5
day                                             22
day_of_year                                    141
day_cos                                  -0.755493
day_sin                                   0.655156
p1_glicko2_rating                      1894.970825
p1_glicko2_rd                            62.629372
p1_glicko2_vol                            0.059135
p2_glicko2_rating              

In [14]:
frances_df.loc[877]

tourney_date                              20230508
tourney_level                                    M
p1_name                          Diego Schwartzman
p2_name                             Matteo Arnaldi
p1_age                                        30.7
p2_age                                        22.2
p1_iso                                          AR
p2_iso                                          IT
winner                                           1
winner_name                         Matteo Arnaldi
year                                          2023
month                                            5
day                                              8
day_of_year                                    127
day_cos                                  -0.577292
day_sin                                   0.816538
p1_glicko2_rating                      1907.092578
p1_glicko2_rd                            62.788667
p1_glicko2_vol                            0.059137
p2_glicko2_rating              

In [8]:
# Check for null values in each row
null_rows = df.isnull().any(axis=1)

# Select rows with null values
rows_with_null = df[null_rows]

print(rows_with_null)

Empty DataFrame
Columns: [tourney_date, tourney_level, p1_name, p2_name, p1_age, p2_age, p1_iso, p2_iso, winner, winner_name, year, month, day, day_of_year, day_cos, day_sin, p1_glicko2_rating, p1_glicko2_rd, p1_glicko2_vol, p2_glicko2_rating, p2_glicko2_rd, p2_glicko2_vol, tourney_level_cond, tourney_datetime, tourney_date_int, 2weeks_ago, semester_ago, year_ago, p1_id, p2_id, winner_id, p1_tour_wins_last_sem, p1_tour_losses_last_sem, p1_tour_wins_last_year, p1_tour_losses_last_year, p1_tour_wins_alltime, p1_tour_losses_alltime, p1_qual/chal_wins_last_sem, p1_qual/chal_losses_last_sem, p1_qual/chal_wins_last_year, p1_qual/chal_losses_last_year, p1_qual/chal_wins_alltime, p1_qual/chal_losses_alltime]
Index: []

[0 rows x 43 columns]
